## osumapper #4: New Map Reader

1.   List item
2.   List item




In [1]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Joe's cell
DRIVE_PATH = 'drive/MyDrive/Cal/Colab/DeepMusicVisualizer'

In [ ]:
# Richard's cell
DRIVE_PATH = 'drive/MyDrive/cs194/DeeperMusicVisualizer'

In [3]:
%cd $DRIVE_PATH

/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer


In [4]:
%cd rhythm

/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer/rhythm


In [5]:
%load_ext autoreload
%autoreload 2

Set the input file string to a timed (having the right BPM/offset) .osu file.

It converts the map/music to Python readable format.

In [6]:
!pip install essentia

     |████████████████████████████████| 12.0MB 3.3MB/s 


In [7]:
%reload_ext autoreload
from act_newmap_prep import *

# input file here! (don't remove the "r" before string)
#file_path = r'../data/beatmaps_57/Faded/beatmap.osu'

# Or use auto timing with music file only!!
from act_timing import *;
music_path = r"../data/beatmaps_57/Faded/audio.mp3"

file_path = get_timed_osu_file(music_path, game_mode=0);
print(file_path)
step4_read_new_map(file_path);

BPM: 89.99308013916016
Offset: 708.2086205482483
Final BPM: 90.0
timing.osu


In [13]:
# file_path = 'timing.osu'
# file_path = 'timing_hardcoded_singletiming.osu'
file_path = 'timing_hardcoded_alltiming.osu'
step4_read_new_map(file_path);

## osumapper #5: Rhythm Predictor

Calculates a map's rhythm based on the music and timing.

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"slider_favor" determines how the model favors sliders against circles, ranges from -1 to 1.<br>
"dist_multiplier" determines the distance snap. ranges from 0 to +∞. Of course 0/+∞ are not advisable.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"slider_max_ticks" determines the max amount of time a slider can slide, ranges from 1 to +∞.

In [14]:
%reload_ext autoreload
from act_rhythm_calc import *

model = step5_load_model("rhythm_model_8_LSTM128_overfit.h5");
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 8, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);

(113, 16, 5)
s (1808, 5) [-1. -1. -1. -1. -1.]
(1808, 11) (113, 16, 11) 8
[-0.9907979]
(1808, 1)
others (1808, 1) [0.]
another (1808, 5)
anoter [0. 0. 0. 0. 0.]
632 notes predicted.
(1808, 1) (1808, 5) (1821,) (1821,) (1808, 11)
[130 297 463 630 797]
[0 1 2 3 4]
384
(393,) (393, 5) (393,) (393,) (393,) (393,) (393,) (393,) (393,)
slider preds [1. 0. 1. 0. 0.]
slider ticks [4 4 8 0 2]
timestamps [ 130 1463 2797 4797 5130]
ticks [ 0  8 16 28 30]
1 16 2797 1.0 0.0 0.0


In [16]:
# this code is buggy so i copied the content of the method and modified it below (added .npz to end of save)
# step5_save_predictions(converted);

objs, predictions, ticks, timestamps, is_slider, is_spinner, is_note_end, sv, slider_ticks, dist_multiplier = converted;

np.savez_compressed("rhythm_data_faded_128_50epochs_hardcoded_alltiming.npz",
    objs = objs,
    predictions = predictions,
    ticks = ticks,
    timestamps = timestamps,
    is_slider = is_slider,
    is_spinner = is_spinner,
    is_note_end = is_note_end,
    sv = sv,
    slider_ticks = slider_ticks,
    dist_multiplier = dist_multiplier);

## osumapper #6: Map flow generator

Generate the final map using a Generative Adversarial Network.

Parameters:

- note_distance_basis: the baseline for distance snap between notes
- max_ticks_for_ds: max number of time ticks (each 1/4) that it uses the distance snap
- next_from_slider_end: use slider end instead of slider head for calculating distance
- box_loss_border, box_loss_value: it's like a barrier on the map edges that bounces off the circles
- divisor, note_group_size: don't change unless you're using a special model built for it
- good_epoch, max_epoch: controls the training time. less time makes it faster but risks less quality
- g_\*, c_\*: hyperparameters used by GAN. No one knows how they work but they mysterically affect the result

In [ ]:
from act_gan import *;

gan_params = {
    "divisor" : 4,
    "good_epoch" : 12,
    "max_epoch" : 30,
    "note_group_size" : 10,
    "g_epochs" : 1,
    "c_epochs" : 1,
    "g_batch" : 50,
    "g_input_size" : 50,
    "c_true_batch" : 140,
    "c_false_batch" : 5,
    "c_randfalse_batch" : 5,
    "note_distance_basis" : 200,
    "next_from_slider_end" : False,
    "max_ticks_for_ds" : 1,
    "box_loss_border" : 0.1,
    "box_loss_value" : 0.4,
    "box_loss_weight" : 1
};

step6_set_gan_params(gan_params);
osu_a, data = step6_run_all();

### Since the generation will take a while...

we can appreciate a nice picture of Cute Sophie!!

<img src="https://i.imgur.com/Ko2wogO.jpg" />

Do a little modding to the map.

Parameters:

- stream_regularizer: fix bad streams. integer for modes (0,1,2,3,4) 0=inactive
- slider_mirror: mirror slider ends if they go outside map area. (0,1) 0=inactive 1=active

In [ ]:
from act_modding import *

modding_params = {
    "stream_regularizer" : 1,
    "slider_mirror" : 1
}

osu_a, data = step7_modding(osu_a, data, modding_params);

Finally, save the data into an .osu file!

In [ ]:
from act_final import *

saved_osu_name = step8_save_osu_file(osu_a, data);

# for taiko mode only (comment out the above line and use below)
# from act_taiko_hitsounds import *
# taiko_hitsounds_params = step8_taiko_hitsounds_set_params(divisor=4, metronome_count=4)
# hitsounds = step8_apply_taiko_hitsounds(osu_a, data, params=taiko_hitsounds_params)
# saved_osu_name = step8_save_osu_file(osu_a, data, hitsounds=hitsounds);

# clean up the folder
step8_clean_up();

If it works alright, you should have a nice .osu file under the folder of these notebooks now!

If it does not work, please tell me the problem so probably I could fix it!

For bug reports and feedbacks either report it on github or use discord: <br>
[https://discord.com/invite/npmSy7K](https://discord.com/invite/npmSy7K)